## Socioeconomic data and TOC entitlements
* Entitlements assigned to census tracts
* Which census tracts (what income levels or median household income) have seen TOC entitlements?

In [1]:
import intake
import numpy as np
import pandas as pd
import geopandas as gpd
import utils

In [2]:
catalog = intake.open_catalog("../catalogs/*.yml")

bucket_name = 'city-planning-entitlements'

## Merge parcels with census tracts crosswalk

In [3]:
crosswalk = pd.read_parquet(f's3://{bucket_name}/data/crosswalk_parcels_tracts.parquet')

In [7]:
parcels = gpd.read_file(
    f"s3://{bucket_name}/gis/intermediate/toc_eligible_parcels_with_entitlements.geojson"
)

parcels['max_tier'] = parcels.groupby('AIN')['TOC_Tier'].transform('max')
parcels = parcels[parcels.TOC_Tier == parcels.max_tier]
parcels = parcels.drop(columns = 'max_tier')

In [17]:
# Join parcels to census tracts
m1 = pd.merge(parcels, crosswalk, on = 'AIN', how = 'inner')
m1 = m1.to_crs({'init':'epsg:4326'})
m1.head()

,AIN,TOC_Tier,zone_class,num_TOC,num_nonTOC,geometry,GEOID,pop
0,2023007025,2,C1,0,1,POINT (-118.60630 34.20252),06037134303,6160
1,2038040006,1,C2,0,1,POINT (-118.62737 34.19253),06037135201,2839
2,2038040011,2,C2,0,1,POINT (-118.62377 34.19251),06037135201,2839
3,2038040012,1,C2,0,2,POINT (-118.62468 34.19179),06037135201,2839
4,2038040013,1,C2,0,1,POINT (-118.62541 34.19124),06037135201,2839


## Merge in TOC Tiers

In [18]:
toc_tiers = utils.reconstruct_toc_tiers_file()
toc_tiers.head()

/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)


,tiers_id,line_id_a,line_id_b,line_name_a,line_name_b,station_id,station_name,geometry,tier_1,tier_2,tier_3,tier_4,mode_a,mode_b,agency_a,agency_b
0,1,705-13108,105-13108,705,105,,,GEOMETRYCOLLECTION (POINT (-118.38161 34.08492...,"MULTIPOLYGON (((-118.36948 34.01327, -118.3699...","MULTIPOLYGON (((-118.23340 34.00814, -118.2330...","MULTIPOLYGON (((-118.22726 34.00397, -118.2272...",GEOMETRYCOLLECTION EMPTY,bus,bus,Metro - Los Angeles,Metro - Los Angeles
1,2,105-13108,705-13108,105,705,,,GEOMETRYCOLLECTION (POINT (-118.38161 34.08492...,"MULTIPOLYGON (((-118.36948 34.01327, -118.3699...","MULTIPOLYGON (((-118.23340 34.00814, -118.2330...","MULTIPOLYGON (((-118.22726 34.00397, -118.2272...",GEOMETRYCOLLECTION EMPTY,bus,bus,Metro - Los Angeles,Metro - Los Angeles
2,3,180-13108,780-13108,180/181,780,,,GEOMETRYCOLLECTION (POINT (-118.31836 34.10169...,"MULTIPOLYGON (((-118.15562 34.15335, -118.1554...","MULTIPOLYGON (((-118.15559 34.14827, -118.1554...","MULTIPOLYGON (((-118.11833 34.14597, -118.1183...",GEOMETRYCOLLECTION EMPTY,bus,bus,Metro - Los Angeles,Metro - Los Angeles
3,4,780-13108,180-13108,780,180/181,,,GEOMETRYCOLLECTION (POINT (-118.31836 34.10169...,"MULTIPOLYGON (((-118.15562 34.15335, -118.1554...","MULTIPOLYGON (((-118.15559 34.14827, -118.1554...","MULTIPOLYGON (((-118.11833 34.14597, -118.1183...",GEOMETRYCOLLECTION EMPTY,bus,bus,Metro - Los Angeles,Metro - Los Angeles
4,5,754-13108,204-13108,754,204,,,GEOMETRYCOLLECTION (POINT (-118.29176 34.09910...,"MULTIPOLYGON (((-118.30202 34.04457, -118.3020...","MULTIPOLYGON (((-118.28571 34.06907, -118.2857...","MULTIPOLYGON (((-118.28868 34.06907, -118.2886...",GEOMETRYCOLLECTION EMPTY,bus,bus,Metro - Los Angeles,Metro - Los Angeles


In [19]:
df = utils.parcels_join_toc_tiers(m1, toc_tiers)

## Merge in commute info

In [20]:
df.head()

/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)


,AIN,TOC_Tier,zone_class,num_TOC,num_nonTOC,geometry,tiers_id,line_id_a,line_id_b,line_name_a,...,tier_1,tier_2,tier_3,tier_4,mode_a,mode_b,agency_a,agency_b,a_rapid,b_rapid
0,2038040006,1,C2,0,1,POINT (-118.62737 34.19253),101.0,152-13108,165-13108,152/353,...,GEOMETRYCOLLECTION EMPTY,"MULTIPOLYGON (((-118.36731 34.19397, -118.3673...",GEOMETRYCOLLECTION EMPTY,GEOMETRYCOLLECTION EMPTY,bus,bus,Metro - Los Angeles,Metro - Los Angeles,False,False
1,2038040006,1,C2,0,1,POINT (-118.62737 34.19253),104.0,152-13108,164-13108,152/353,...,GEOMETRYCOLLECTION EMPTY,"MULTIPOLYGON (((-118.36731 34.18672, -118.3673...",GEOMETRYCOLLECTION EMPTY,GEOMETRYCOLLECTION EMPTY,bus,bus,Metro - Los Angeles,Metro - Los Angeles,False,False
2,2038040012,1,C2,0,2,POINT (-118.62468 34.19179),101.0,152-13108,165-13108,152/353,...,GEOMETRYCOLLECTION EMPTY,"MULTIPOLYGON (((-118.36731 34.19397, -118.3673...",GEOMETRYCOLLECTION EMPTY,GEOMETRYCOLLECTION EMPTY,bus,bus,Metro - Los Angeles,Metro - Los Angeles,False,False
3,2038040012,1,C2,0,2,POINT (-118.62468 34.19179),104.0,152-13108,164-13108,152/353,...,GEOMETRYCOLLECTION EMPTY,"MULTIPOLYGON (((-118.36731 34.18672, -118.3673...",GEOMETRYCOLLECTION EMPTY,GEOMETRYCOLLECTION EMPTY,bus,bus,Metro - Los Angeles,Metro - Los Angeles,False,False
4,2038040013,1,C2,0,1,POINT (-118.62541 34.19124),101.0,152-13108,165-13108,152/353,...,GEOMETRYCOLLECTION EMPTY,"MULTIPOLYGON (((-118.36731 34.19397, -118.3673...",GEOMETRYCOLLECTION EMPTY,GEOMETRYCOLLECTION EMPTY,bus,bus,Metro - Los Angeles,Metro - Los Angeles,False,False
